In [3]:
import pandas as pd

In [4]:
import altair as alt

In [5]:
sessions=pd.read_csv("sessions.csv")
sessions

,hashedEmail,start_time,end_time,original_start_time,original_end_time
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,30/06/2024 18:12,30/06/2024 18:24,1.719770e+12,1.719770e+12
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,17/06/2024 23:33,17/06/2024 23:46,1.718670e+12,1.718670e+12
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,25/07/2024 17:34,25/07/2024 17:57,1.721930e+12,1.721930e+12
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,25/07/2024 03:22,25/07/2024 03:58,1.721880e+12,1.721880e+12
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,25/05/2024 16:01,25/05/2024 16:12,1.716650e+12,1.716650e+12
...,...,...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,10/05/2024 23:01,10/05/2024 23:07,1.715380e+12,1.715380e+12
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,01/07/2024 04:08,01/07/2024 04:19,1.719810e+12,1.719810e+12
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,28/07/2024 15:36,28/07/2024 15:57,1.722180e+12,1.722180e+12
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,25/07/2024 06:15,25/07/2024 06:22,1.721890e+12,1.721890e+12


In [6]:
sessions[["start_day","start_month","start_year"]]= sessions["start_time"].str.split("/", expand=True)
sessions[["start_year", "start_hour_minute"]]= sessions["start_year"].str.split(" ", expand=True)
sessions[["end_day","end_month","end_year"]]= sessions["end_time"].str.split("/", expand=True)
sessions[["end_year", "end_hour_minute"]]= sessions["end_year"].str.split(" ", expand=True)
sessions = sessions.drop(columns = ["start_time","end_time"])
sessions

,hashedEmail,original_start_time,original_end_time,start_day,start_month,start_year,start_hour_minute,end_day,end_month,end_year,end_hour_minute
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,1.719770e+12,1.719770e+12,30,06,2024,18:12,30,06,2024,18:24
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,1.718670e+12,1.718670e+12,17,06,2024,23:33,17,06,2024,23:46
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,1.721930e+12,1.721930e+12,25,07,2024,17:34,25,07,2024,17:57
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,1.721880e+12,1.721880e+12,25,07,2024,03:22,25,07,2024,03:58
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,1.716650e+12,1.716650e+12,25,05,2024,16:01,25,05,2024,16:12
...,...,...,...,...,...,...,...,...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,1.715380e+12,1.715380e+12,10,05,2024,23:01,10,05,2024,23:07
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,1.719810e+12,1.719810e+12,01,07,2024,04:08,01,07,2024,04:19
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,1.722180e+12,1.722180e+12,28,07,2024,15:36,28,07,2024,15:57
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,1.721890e+12,1.721890e+12,25,07,2024,06:15,25,07,2024,06:22


In [7]:
dropped_sessions=sessions.drop(columns=["start_year","end_year","hashedEmail"])
dropped_sessions

,original_start_time,original_end_time,start_day,start_month,start_hour_minute,end_day,end_month,end_hour_minute
0,1.719770e+12,1.719770e+12,30,06,18:12,30,06,18:24
1,1.718670e+12,1.718670e+12,17,06,23:33,17,06,23:46
2,1.721930e+12,1.721930e+12,25,07,17:34,25,07,17:57
3,1.721880e+12,1.721880e+12,25,07,03:22,25,07,03:58
4,1.716650e+12,1.716650e+12,25,05,16:01,25,05,16:12
...,...,...,...,...,...,...,...,...
1530,1.715380e+12,1.715380e+12,10,05,23:01,10,05,23:07
1531,1.719810e+12,1.719810e+12,01,07,04:08,01,07,04:19
1532,1.722180e+12,1.722180e+12,28,07,15:36,28,07,15:57
1533,1.721890e+12,1.721890e+12,25,07,06:15,25,07,06:22


In [8]:
players=pd.read_csv("players.csv")
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


In [9]:
dropped_players=players.drop(columns=["individualId","organizationName","hashedEmail"])
dropped_players

,experience,subscribe,played_hours,name,gender,age
0,Pro,True,30.3,Morgan,Male,9
1,Veteran,True,3.8,Christian,Male,17
2,Veteran,False,0.0,Blake,Male,17
3,Amateur,True,0.7,Flora,Female,21
4,Regular,True,0.1,Kylie,Male,21
...,...,...,...,...,...,...
191,Amateur,True,0.0,Bailey,Female,17
192,Veteran,False,0.3,Pascal,Male,22
193,Amateur,False,0.0,Dylan,Prefer not to say,17
194,Amateur,False,2.3,Harlow,Male,17


In [22]:
players_plot_bar1 = alt.Chart(dropped_players).mark_bar().encode(
    x = alt.X("experience").title("Experience of Players"),
    y = alt.Y("played_hours").title("Played Hours")
)
players_plot_bar1

alt.Chart(...)

In [21]:
players_plot_scatter = alt.Chart(dropped_players).mark_point().encode(
    x = alt.X("age").title("Age of Player"),
    y= alt.Y("played_hours").title("Played Hours"),
    color = alt.Color("experience").title("Experience").scale(scheme = "set1")
)
players_plot_scatter

alt.Chart(...)

In [25]:
players_plot_bar2 = alt.Chart(dropped_players).mark_bar().encode(
    x = alt.X("subscribe").title("Players Subcribed"),
    y = alt.Y("played_hours").title("Played Hours"),
    color = alt.Color("experience").title("Experience").scale(scheme = "set1")
)
players_plot_bar2

alt.Chart(...)